In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/homesite-data/RevisedHomesiteTest1.csv
/kaggle/input/homesite-data/RevisedHomesiteTrain1.csv


In [2]:
!pip install vecstack

In [3]:
from vecstack import stacking
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.preprocessing import OneHotEncoder
from collections import Counter

In [4]:
train_data = pd.read_csv("/kaggle/input/homesite-data/RevisedHomesiteTrain1.csv")
test_data = pd.read_csv("/kaggle/input/homesite-data/RevisedHomesiteTest1.csv")

print(train_data.shape)
print(test_data.shape)
print(train_data.head()) 

(65000, 596)
(173836, 596)
   CoverageField11A  CoverageField11B  CoverageField1A  CoverageField1B  \
0                 2                 1               17               23   
1                 5                 9                6                8   
2                 4                 6                7               12   
3                15                23                3                2   
4                 4                 6                8               13   

   CoverageField2A  CoverageField2B  CoverageField3A  CoverageField3B  \
0               17               23               15               22   
1                6                8                5                7   
2                7               12                6               10   
3                3                2                2                2   
4                8               13                7               11   

   CoverageField4A  CoverageField4B  ...  PropertyField38_N  \
0               16  

In [ ]:
fig = px.histogram(train_data, x='QuoteConversion_Flag', title='Histogram QuoteConversion_Flag')

# Show the plot
fig.show()

In [7]:
train_data.isnull().sum().sort_values(ascending=False)

CoverageField11A        0
PersonalField17_YU      0
PersonalField17_YW      0
PersonalField17_YX      0
PersonalField17_YY      0
                       ..
PersonalField64         0
PersonalField65         0
PersonalField66         0
PersonalField67         0
QuoteConversion_Flag    0
Length: 596, dtype: int64

In [8]:
train_data_columns = train_data.columns
train_data_columns

Index(['CoverageField11A', 'CoverageField11B', 'CoverageField1A',
       'CoverageField1B', 'CoverageField2A', 'CoverageField2B',
       'CoverageField3A', 'CoverageField3B', 'CoverageField4A',
       'CoverageField4B',
       ...
       'PropertyField38_N', 'PropertyField38_Y', 'GeographicField63_ ',
       'GeographicField63_N', 'GeographicField63_Y', 'GeographicField64_CA',
       'GeographicField64_IL', 'GeographicField64_NJ', 'GeographicField64_TX',
       'QuoteConversion_Flag'],
      dtype='object', length=596)

In [9]:
x_train = train_data[7500:]
x_test = train_data[:7500]
y_train1 = x_train["QuoteConversion_Flag"]
y_test1 = x_test["QuoteConversion_Flag"]
x_train1 = x_train.iloc[:, :-1]
x_test1 = x_test.iloc[:, :-1]

In [10]:
clf = DecisionTreeClassifier()
clf.fit(x_train1,y_train1)
clf_predict=clf.predict(x_test1)
print("Accuracy Score (training) for Decision Tree:{0:2f}".format(clf.score(x_test1,y_test1)))
print("Confusion Matrix for Decision Tree")
print(confusion_matrix(y_test1,clf_predict))

Accuracy Score (training) for Decision Tree:0.858800
Confusion Matrix for Decision Tree
[[5436  629]
 [ 430 1005]]


In [11]:
# Hyperparameter tuning done for decision tree classifier
parameters = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'min_samples_split': range(2, 20),
    'min_samples_leaf': range(1, 10),
    'max_depth': range(1, 20, 2),
    'max_features': [None, 'sqrt', 'log2']
}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=50)
clf_random.fit(x_train1, y_train1)
grid_parm=clf_random.best_params_
print(grid_parm)

# Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
clf = DecisionTreeClassifier(**grid_parm)
clf.fit(x_train1,y_train1)
clf_predict = clf.predict(x_test1)

# Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (training) after hypertuning for Decision Tree:{0:6f}".format(clf.score(x_test1,y_test1)))
print("Confusion Matrix after hypertuning for Decision Tree")
print(confusion_matrix(y_test1,clf_predict))
print("=== Classification Report ===")
print(classification_report(y_test1,clf_predict))

#get cross-validation report
clf_cv_score = cross_val_score(clf, x_train1, y_train1, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Decision Tree: ",clf_cv_score.mean())

{'splitter': 'best', 'min_samples_split': 13, 'min_samples_leaf': 4, 'max_features': None, 'max_depth': 9, 'criterion': 'gini'}
accuracy Score (training) after hypertuning for Decision Tree:0.913733
Confusion Matrix after hypertuning for Decision Tree
[[5984   81]
 [ 566  869]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.91      0.99      0.95      6065
           1       0.91      0.61      0.73      1435

    accuracy                           0.91      7500
   macro avg       0.91      0.80      0.84      7500
weighted avg       0.91      0.91      0.91      7500

=== All AUC Scores ===
[0.93673131 0.93264867 0.94135277 0.94220763 0.94184329 0.94564258
 0.94017839 0.93933516 0.94414576 0.93531724]


=== Mean AUC Score ===
Mean AUC Score - Decision Tree:  0.9399402808168492


In [13]:
# Random Forest Model

rfc = RandomForestClassifier()
rfc.fit(x_train1, y_train1)
rfc_predict=rfc.predict(x_test1)
print("Accuracy Score (training) for RandomForest:{0:2f}".format(rfc.score(x_test1,y_test1)))
print("Confusion Matrix for Random Forest:")
print(confusion_matrix(y_test1,rfc_predict))

Accuracy Score (training) for RandomForest:0.903867
Confusion Matrix for Random Forest:
[[5971   94]
 [ 627  808]]


In [14]:
# MultiLayer Perceptron Model

mlp = MLPClassifier(max_iter=100)
mlp.fit(x_train1, y_train1)
mlp_predict=mlp.predict(x_test1)
print("Accuracy Score (training) for MultiLayer Perceptron:{0:6f}".format(mlp.score(x_test1,y_test1)))
print("Confusion Matrix for MultiLayer Perceptron:")
print(confusion_matrix(y_test1,mlp_predict))

Accuracy Score (training) for MultiLayer Perceptron:0.673333
Confusion Matrix for MultiLayer Perceptron:
[[4292 1773]
 [ 677  758]]


In [15]:
# K-Nearest Neighbor Model
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)

neigh.fit(x_train1, y_train1)
neigh_predict=neigh.predict(x_test1)
print("Accuracy Score (training) for KNeighborsClassifier:{0:2f}".format(neigh.score(x_test1,y_test1)))
print("Confusion Matrix for KNeighborsClassifier:")
print(confusion_matrix(y_test1,neigh_predict))

Accuracy Score (training) for KNeighborsClassifier:0.753867
Confusion Matrix for KNeighborsClassifier:
[[5535  530]
 [1316  119]]


In [16]:
# Linear Support Vector Machine Model

from sklearn.svm import LinearSVC 
linsvm = LinearSVC(max_iter=300) 
linsvm.fit(x_train1, y_train1) 
linsvm_predict=linsvm.predict(x_test1) 
print("Accuracy Score (training) for Linear SVM Classifier:{0:6f}".format(linsvm.score(x_test1,y_test1))) 
print("Confusion Matrix for Linear SVM Classifier:") 
print(confusion_matrix(y_test1,linsvm_predict))

Accuracy Score (training) for Linear SVM Classifier:0.809200
Confusion Matrix for Linear SVM Classifier:
[[5954  111]
 [1320  115]]


/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



In [17]:
sampling_strategy = 1

# Create the SMOTE object
sm = SMOTE(sampling_strategy=sampling_strategy)

# Resample the dataset
X_res, y_res = sm.fit_resample(x_train1, y_train1)

print('Resampled dataset shape %s' % Counter(y_res))

Resampled dataset shape Counter({0: 46673, 1: 46673})


In [37]:
test_data.drop(['GeographicField64'], axis=1, inplace=True)

#### Ensemble Methods

In [38]:
models = [KNeighborsClassifier(), MLPClassifier(), LinearSVC(), RandomForestClassifier(), DecisionTreeClassifier() ]
      
S_Train, S_Test = stacking(models,                   
                           X_res, y_res, test_data,   
                           regression=False, 
                           mode='oof_pred_bag', 
                           needs_proba=False,
                           save_dir=None, 
                           metric=accuracy_score,
                           n_folds=4, 
                           stratified=True,
                           shuffle=True,
                           random_state=0,    
                           verbose=2)

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [5]

model  0:     [KNeighborsClassifier]
    fold  0:  [0.72481467]
    fold  1:  [0.72322921]
    fold  2:  [0.72707405]
    fold  3:  [0.73015941]
    ----
    MEAN:     [0.72631933] + [0.00260426]
    FULL:     [0.72631929]

model  1:     [MLPClassifier]
    fold  0:  [0.85229464]
    fold  1:  [0.83793975]
    fold  2:  [0.89586904]
    fold  3:  [0.91875214]
    ----
    MEAN:     [0.87621389] + [0.03253025]
    FULL:     [0.87621323]

model  2:     [LinearSVC]


/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



    fold  0:  [0.50049278]


/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



    fold  1:  [0.55658397]


/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



    fold  2:  [0.50647069]


/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



    fold  3:  [0.67308022]
    ----
    MEAN:     [0.55915691] + [0.06928647]
    FULL:     [0.55915626]

model  3:     [RandomForestClassifier]
    fold  0:  [0.94065218]
    fold  1:  [0.94103784]
    fold  2:  [0.94266370]
    fold  3:  [0.93747857]
    ----
    MEAN:     [0.94045807] + [0.00187856]
    FULL:     [0.94045808]

model  4:     [DecisionTreeClassifier]
    fold  0:  [0.92106955]
    fold  1:  [0.91952693]
    fold  2:  [0.92012341]
    fold  3:  [0.91939493]
    ----
    MEAN:     [0.92002870] + [0.00066064]
    FULL:     [0.92002871]

